In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

# Langsmith Tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

# Data Ingestion (Scraping the data)

In [2]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [4]:
# Mise sosu forme de document
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Releases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up bi

# Split the document

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content="Manage billing in your account - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Releases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up bi

# Stock the documents in vector store db as Embedding

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()

In [7]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents,embeddings)

# Query from vector db

In [8]:
query="Langsmith has two usage limits:total traces and extended"
result=vectorstoredb.similarity_search(query)
print(result[0].page_content)



For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.
​Configure trace tier distrubution
LangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.


# Retrieval Chain, Documents Chain

In [9]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000019AD9363EE0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019AD9363670> root_client=<openai.OpenAI object at 0x0000019AD9363730> root_async_client=<openai.AsyncOpenAI object at 0x0000019AD9363E50> model_name='gpt-4o' openai_api_key=SecretStr('**********') openai_proxy=''


### Message que nous allons envoyer au modèle

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only the provided context:  
<context>
{context}
</context>
    
    """
)

# On demande au llm quoi faire
# {context} -> l’endroit où les documents trouvés par le retriever seront insérés.
# Empêche les hallucinations car le modèle sait qu'il doit rester dans le contexte fourni

document_chain=create_stuff_documents_chain(llm,prompt)  # crée une mini-pipeline prête pour le RAG

# “Stuff” = on met tous les documents dans {context}

# Donc cette ligne crée une chaîne qui :

# prend tes documents

# les concatène dans {context}

# envoie le tout au LLM

# retourne la réponse

In [25]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only the provided context:  \n<context>\n{context}\n</context>\n    \n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019AD9363EE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019AD9363670>, root_client=<openai.OpenAI object at 0x0000019AD9363730>, root_async_client=<openai.AsyncOpenAI object at 0x0000019AD9363E50>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

### Exemple avec un document créé

In [26]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"Langsmith has two usage limits:total traces and extended",  # "input" = la question de l’utilisateur
    "context":[Document(page_content="Langsmith has two usage limits:total traces and extended traces. These correspond to the two metrics we-ve been tracking on our usage graph. ")]  
    # "context" = les documents que tu veux donner au LLM
})

# La chain va :

# remplacer {context} dans le prompt par tes documents

# construire un message final

# l’envoyer au LLM

# me renvoyer la réponse

'Langsmith has two usage limits: total traces and extended traces. These limits correspond to the two metrics being tracked on their usage graph.'

### Retrieval

In [27]:
# Input ---> Retriever ---> vectorstoredb

from langchain.chains import create_retrieval_chain

retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [28]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019AD93613C0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only the provided context:  \n<context>\n{context}\n</context>\n    \n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019AD9363EE0>, async_client=<openai.resources.chat.completions.completions.AsyncCo

In [29]:
# Get de response from llm

response=retrieval_chain.invoke({"input": "Langsmith has two usage limits:total traces and extended"})

In [30]:
response

{'input': 'Langsmith has two usage limits:total traces and extended',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'),
  Document(metadata={'sou

In [31]:
response['answer']

'For organizations with multiple workspaces, how are free traces incorporated into cost calculations according to the context?\n\nLangSmith incorporates the free traces into the cost calculation of the first workspace only. However, the free traces can be "consumed" by any workspace within the organization. While workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.'

In [32]:
response['context'] # L'endroit où a été pris la réponse

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'ti